# Collab groupe 2

##Chargement base de donnée et bibliotèques

Chargement des bibliotèques :

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
#from sklearn.metrics import root_mean_squared_error

Chargement des données sur la base de donnée :

In [ ]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT  "primaryTitle", "titleType", "isAdult", "startYear", "runtimeMinutes", "genres", "averageRating", "directors", "writers", "actor", "producer", "cinematographer", "composer", "editor", "production_designer", "self", "archive_footage", "archive_sound"
from "castview"
where "runtimeMinutes" Is NOT null and "averageRating" is NOT NULL and "genres" is NOT NULL and "startYear" is NOT NULL and "isAdult" is NOT NULL  and "directors" is NOT NULL and  "writers" is NOT NULL  and  "actor" is NOT NULL  and  "producer" is NOT NULL
limit 100000;
"""

#and "titleType" = 'movie'

df = pd.read_sql(SQL, engine)
engine.dispose()
df

## Analyse des données

Répartition des données

In [ ]:
df.info()

Répartition des ratings

In [ ]:
df['averageRating'].hist()
plt.title('Distribution des Rating ')
plt.xlabel('averageRating')
plt.ylabel('Nombre de Film')

Création d'un diagramme en boîte qui montre la distribution des durées des films (runtimeMinutes) pour chaque type de titre (titleType).

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='titleType', y='runtimeMinutes', data=df)
plt.title('Distribution des runtimeMinutes par Type (titleType)')
plt.xlabel('titleType')
plt.ylabel('runtimeMinutes (min)')
plt.show()

## Création des fonctions

Convertion liste en une chaine de caractère

In [ ]:
def liste_en_texte(lst):
    if isinstance(lst, list):
        return ' '.join(lst)
    else:
        return lst

Nettoyage des données

In [ ]:
def cleanText(df):
    df.fillna('missing', inplace=True)
    df=df.str.replace(',', ' ')
    return df

##Création du DF avec les colonnes choisies

In [ ]:
columns_to_clean = ['primaryTitle', 'titleType', 'genres', 'directors', 'writers',
                    'actor', 'producer', 'cinematographer', 'composer', 'editor',
                    'production_designer', 'self', 'archive_footage', 'archive_sound']

for column in columns_to_clean:
    df[column] = cleanText(df[column])

In [ ]:
df

##Entrainement du modèle

Séparation des caractéristiques et de la cible et des données d'entraînement et de test avec y comme target et X les features

In [ ]:

X = df.drop(columns=["averageRating"])
y = df["averageRating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Définition des colonnes numériques, textuelles et de description

In [ ]:
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = ['titleType']

Création des transformers pour les colonnes numériques, booléennes, textuelles et de description et du ColumnTransformer

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1))

text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    ('vect', CountVectorizer(decode_error='ignore', analyzer='word')) #max_features=1000, analyzer="word"
])
tfidf_transformer = Pipeline([
    ('tf_idf', TfidfVectorizer(decode_error='ignore',analyzer='word')) #max_features=1000, sublinear_tf=True
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', text_transformer, text_features), # c'est titleType

        ('title', Vect_transformer, 'primaryTitle'),

        ('genres', tfidf_transformer, 'genres'),
        ('producer', tfidf_transformer, 'producer'),
        ('directors', tfidf_transformer, 'directors'),
        ('writers', tfidf_transformer, 'writers'),
        ('actor', tfidf_transformer, 'actor'),

        ('cinematographer', tfidf_transformer, 'cinematographer'),
        ('composer', tfidf_transformer, 'composer'),
        ('editor', tfidf_transformer, 'editor'),
        ('production_designer', tfidf_transformer, 'production_designer'),
        ('selfy', tfidf_transformer, 'self'),
        ('archive_footage', tfidf_transformer, 'archive_footage'),
        ('archive_sound', tfidf_transformer, 'archive_sound')


    ])
preprocessor

Création des pipelines pour chaque modèle et leur paramètres.
Affichage des différents scoring.

In [ ]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 5.0,10]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 5.0,10]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 5.0,10], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50, 100]},# None,
    'Gradient Boosting Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50, 100]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


## Résultats et évaluations

Création du GridSearch pour avoir les meilleurs scores, modèles, paramètres.

In [ ]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):


  # Grid search
  grid = GridSearchCV(pipeline, parameters,  scoring=scoring, refit='RMSE', cv=cv, n_jobs =-1, verbose = 0)#, error_score='raise'

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

Création de la fonction pour l'affichage des résultats.

In [ ]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")




Affichage des résultats pour chaque modèles et les meilleurs paramètres associés.

In [ ]:
# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)

    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")

'''